<img align="center" src="https://raw.githubusercontent.com/FUlyankin/Parsers/master/images%20/cats.jpg" height="1200" width="1200"> 

# <center> Грабёж, разбой и другие незаконные деяния с помощью Python. <br> <br> Пауки, Scrapy и Grab  </center>

Первое, что хотелось бы сказать это то, что сегодня будет довольно сложно...

---------


## Объектно-ориентированное программирование (ООП) 

Существует довольно много различных парадигм (грубо говоря стилей) программирования. Можно сказать, что мы с вами до этого момента занимались функциональным программированием. Мы представляли программу в форме последовательности функций, поочерёдный вызов которых давал нам ответ. 

Совершенно другой парадигмой является ООП. В ООП программа представляется в виде взаимодействующих объектов. Сразу же взять и проникнуться этой парадигмой, если вы до этого никогда с ней не встречались, довольно сложно. Однако мы попробуем сделать первый приступ на эту страшную штуку. 

Основными понятиями ООП являются **объекты** и **классы**. Класс — это тип, который описывает устройство объектов. Объект — это один экземпляр класса. Например, бабочки это класс. Бабочка-капустница и бабочка-махаон разные объекты этого класса. 

В Python всё является объектами — и строки, и списки, и словари, и вообще всё. Например, мы можем подгрузить из пакета `sklearn` класс для линейных регрессий, а после создать два объекта: регрессию с константой и регрессию без константы. 

In [45]:
from sklearn.linear_model import LinearRegression

regr_const = LinearRegression(fit_intercept=True)
regr_no_const = LinearRegression(fit_intercept=False)


Две созданные регрессии обладают одними и теми же свойствами. Их можно обучит на каких-нибудь данных, по ним можно что-нибдь спрогнозировать и так далее. Но между ними будет одно отличие — константа. 

Если нам нужно создать много похожих, но разных объектов, к каждому из которых нам хотелось бы применять одни и те функции, то ООП нам подходит! 

Кроме того, ООП позволяет контактировать разным объектам между собой, передовать друг-другу какие-то свойства и даже порождать наследников. Напишем собственный класс.

Наш класс это лифты.

In [60]:
class Elevator:
    """ Простой класс для создания лифтов """
    # Переменная класса. Сколько людей было перевезено ВСЕМИ лифтами
    # Она будет изменяться при использовании каждого объекта
    people_lifted = 0
 
    # Инициализация. Эта функция всегда вызывается при создании объекта
    # В отличие от переменной people_lifted, переменные, которые находятся в 
    # инициализации будут изменяться отдельно для каждого объекта
    def __init__(self,name):
        self.name = name  # Каждый раз, когда мы создаём объект, мы даём ему какое-то имя
        self.people_lifted = 0 # Каждый раз, когда мы создаём объект, переменная, отвечающая за коилчество
                               # людей в нём принимает нулевое значение
 
    # Метод для перевозки людей
    def lift(self):
        print ("{} лифт получил человека".format(self.name))
        # Увеличиваем количество людей перевезенных ЭТИМ лифтом
        self.people_lifted += 1
        # Увеличиваем количество людей перевезенных ВСЕМИ лифтами
        Elevator.people_lifted += 1
 
    # Метод печатающий информацию о конкретном лифте
    def info(self):
        print (self.name, "лифт везёт", self.people_lifted, "из", Elevator.people_lifted)

        
# Обычно первый аргумент в имени метода — self. Как говорит автор языка Гвидо Ван Россум,
# это не более чем соглашение: имя self не имеет абсолютно никакого специального значения.
# self полезен для того, чтобы обращаться к другим атрибутам класса! 

Создадим несколько экземпляров нашего класса. Пусть один лифт будет грузовым, а второй пассажирским.

In [61]:
elevator_1 = Elevator("Пассажирский")
elevator_2 = Elevator("Грузовой")

Как мы с вами помним, чтобы получить доступ к методам класса, нужно поставить точку. 

In [62]:
elevator_1.lift() # Везём человека в пассажирском лифте

Пассажирский лифт получил человека


In [63]:
# Везем двоих человек в грузовом лифте
elevator_2.lift()
elevator_2.lift()

Грузовой лифт получил человека
Грузовой лифт получил человека


In [64]:
# Получаем информацию по лифтам
elevator_1.info()
elevator_2.info()

Пассажирский лифт везёт 1 из 3
Грузовой лифт везёт 2 из 3


Про ООП можно говорить очень много и очень долго. Но на данный момент, это всё, что нам понадобится.

## Паук плетёт узор раз

В качестве цели для парсинга мы для начала возьмём учебный сайт с книгами, http://books.toscrape.com/. Как мы с вами выяснили на самой первой паре, каждый сайт представляет из себя HTML-дерево, которое можно обходить с помощью специальных языков. Для того, чтобы найти информацию на сайте, мы можем пойти двумя путями:

* Путь настоящего мужика (если вы девушка, то настоящей сильной независимой девушки): открыть на сайте правой кнопкой мыши inspect и найти HTML-код нужного нам элемента. 
* Путь ленивого балбеса: заходим в магазин расширений для браузера и находим там [SelectorGadget,](https://chrome.google.com/webstore/detail/selectorgadget/mhjhnkcfbdhnjickkkdbjoemdmbfginb?hl=en-US) устанавливаем его. Теперь нам открыты все пути! Справа, около адресной строки у браузера появилась лупа. 

Перейдём на сайт, который является нашей целью и попробуем эту лупу протестировать.

Тыкаем лупой на какой-нибудь элемент, он обводится зелёным и нам предлагается селектор, с помощью которого мы можем этот элемент найти. Все элементы, которые можно найти по этому селектору окрашиваются в жёлтый цвет. Если какой-то из этих жёлтых элементов нас не интересует, окрашиваем его в красный. Предлагаемый селектор уточняется. Удобно? Удобно! Напишем парсер ровно так, как мы это делали раньше. Попытаемся вытащить все ссылки со страницы. 

![картинка с лупой]( ) 

In [33]:
# Раньше: 
import requests               # Наш пакет для реквестов
from bs4 import BeautifulSoup # Красивый суп

response = requests.get('http://books.toscrape.com/') # На любой реквест всегда найдётся 200-респонс!
html = response.content                               # Достаём из респонса контент
soup = BeautifulSoup(html,'html.parser')              # Окрасивосупиваем контент

hrefs = [ ]   # Пустой вектор для записи хрефов
for item in soup.select('.product_pod a'): # Находим в супе по селектору нужные места
    hrefs.append(item.attrs['href']) # Вытаскиваем из каждого места хреф
print(len(hrefs))
hrefs[:5]

40


['catalogue/a-light-in-the-attic_1000/index.html',
 'catalogue/a-light-in-the-attic_1000/index.html',
 'catalogue/tipping-the-velvet_999/index.html',
 'catalogue/tipping-the-velvet_999/index.html',
 'catalogue/soumission_998/index.html']

Кстати говоря, вывод поиска в супе по селектору выглядит как-то вот так:

In [80]:
soup.select('.product_pod a')[:5]

[<a href="catalogue/sapiens-a-brief-history-of-humankind_996/index.html"><img alt="Sapiens: A Brief History of Humankind" class="thumbnail" src="media/cache/be/a5/bea5697f2534a2f86a3ef27b5a8c12a6.jpg"/></a>,
 <a href="catalogue/sapiens-a-brief-history-of-humankind_996/index.html" title="Sapiens: A Brief History of Humankind">Sapiens: A Brief History ...</a>,
 <a href="catalogue/the-requiem-red_995/index.html"><img alt="The Requiem Red" class="thumbnail" src="media/cache/68/33/68339b4c9bc034267e1da611ab3b34f8.jpg"/></a>,
 <a href="catalogue/the-requiem-red_995/index.html" title="The Requiem Red">The Requiem Red</a>,
 <a href="catalogue/the-dirty-little-secrets-of-getting-your-dream-job_994/index.html"><img alt="The Dirty Little Secrets of Getting Your Dream Job" class="thumbnail" src="media/cache/92/27/92274a95b7c251fea59a2b8a78275ab4.jpg"/></a>]

Эту команду мы никогда раньше не использовали, но её вывод для нас довольно привычен. Её работа чем-то напоминает работу метода `findAll`. Можно продолжить развивать код как раньше и написать поисковик информации по каждой ссылке. 

Эх! Мы с вами уже бывалые грабители (или мирные собиратели данных). Пришло время изучить новый инструмент для грабежа (собирания), а именно  `Scrapy`. `Scrapy` – одна из наиболее популярных библиотек для парсинга. Она включает в себя довольно многго разных функциональных возможностей и освобождает юзера от самостоятельного прописывания многих функций. Например, когда мы самостоятельно парсим все ссылки, переодически мы просто не находим какую-то информацию на сайте и код выдаёт нам ошибку. Чтобы избежать этой ошибки, мы прописываем исключение через `try – except`. В `Scrapy` это делать не нужно. 

Самое главное отличие `Scrapy` от других пакетов состоит в том, что нам придётся запускать пауков из консоли. Существуют пути обхода этого отличия, позволяющие запускать пауков прямо из скрипта, но о них мы поговорим ближе к концу блокнота. 

Устанавливаем пакет для создания пауков, `pip3 install scrapy`. С гайдом по установке можно ознакомиться в [документации.](https://doc.scrapy.org/en/latest/intro/install.html) Обратите внимание, что у разных систем есть [свои особенности установки.](https://doc.scrapy.org/en/latest/intro/install.html#intro-install-platform-notes)

Для начала создадим базовый код для нашего робота. У каждого паука есть два обязательных атрибута: имя и список ссылок, которые нужно проанализировать. 

In [37]:
import scrapy

class BookSpider(scrapy.Spider):
    name = 'books'
    start_urls = ['http://books.toscrape.com/']

Создадим на основе класса `scrapy.Spyder` новый класс пауков. Назовём их книжными пауками. Каждый книжный паук будет обладать именем books и списком ссылок, который он должен будет пройти. Можно посмотреть на эти атрибуты.

In [38]:
BookSpider.start_urls

['http://books.toscrape.com/']

In [39]:
BookSpider.name

'books'

Отлично! Сохраним нашего паука в отдельном файлике под названием books.py и с консоли запустим его. Обычно, чтобы запустить какой-то питоний код, нужно использовать команду `python path_do_coda/nash_super_code.py`. Пакет предлагает нам свой более удобный способ запускать коды: `scrapy runspider nash_super_code.py`.

Попробуем запустить код. В консоли появится сообщение следующего вида: 

````
2017-05-25 03:48:48 [scrapy.utils.log] INFO: Scrapy 1.4.0 started (bot: scrapybot)
2017-05-25 03:48:48 [scrapy.utils.log] INFO: Overridden settings: {'SPIDER_LOADER_WARN_ONLY': True}
2017-05-25 03:48:48 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.logstats.LogStats',
 'scrapy.extensions.memusage.MemoryUsage']
2017-05-25 03:48:48 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'scrapy.downloadermiddlewares.redirect.RedirectMiddleware',
 'scrapy.downloadermiddlewares.cookies.CookiesMiddleware',
 'scrapy.downloadermiddlewares.httpproxy.HttpProxyMiddleware',
 'scrapy.downloadermiddlewares.stats.DownloaderStats']
2017-05-25 03:48:48 [scrapy.middleware] INFO: Enabled spider middlewares:
['scrapy.spidermiddlewares.httperror.HttpErrorMiddleware',
 'scrapy.spidermiddlewares.offsite.OffsiteMiddleware',
 'scrapy.spidermiddlewares.referer.RefererMiddleware',
 'scrapy.spidermiddlewares.urllength.UrlLengthMiddleware',
 'scrapy.spidermiddlewares.depth.DepthMiddleware']
2017-05-25 03:48:48 [scrapy.middleware] INFO: Enabled item pipelines:
[]
2017-05-25 03:48:48 [scrapy.core.engine] INFO: Spider opened
2017-05-25 03:48:48 [scrapy.extensions.logstats] INFO: Crawled 0 pages (at 0 pages/min), scraped 0 items (at 0 items/min)
2017-05-25 03:48:48 [scrapy.extensions.telnet] DEBUG: Telnet console listening on 127.0.0.1:6023
2017-05-25 03:48:49 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://books.toscrape.com/> (referer: None)
2017-05-25 03:48:49 [scrapy.core.scraper] ERROR: Spider error processing <GET http://books.toscrape.com/> (referer: None)
Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/twisted/internet/defer.py", line 653, in _runCallbacks
    current.result = callback(current.result, *args, **kw)
  File "/usr/local/lib/python3.5/dist-packages/scrapy/spiders/__init__.py", line 90, in parse
    raise NotImplementedError
NotImplementedError
2017-05-25 03:48:49 [scrapy.core.engine] INFO: Closing spider (finished)
2017-05-25 03:48:49 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 216,
 'downloader/request_count': 1,
 'downloader/request_method_count/GET': 1,
 'downloader/response_bytes': 5900,
 'downloader/response_count': 1,
 'downloader/response_status_count/200': 1,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2017, 5, 25, 0, 48, 49, 310862),
 'log_count/DEBUG': 2,
 'log_count/ERROR': 1,
 'log_count/INFO': 7,
 'memusage/max': 49901568,
 'memusage/startup': 49901568,
 'response_received_count': 1,
 'scheduler/dequeued': 1,
 'scheduler/dequeued/memory': 1,
 'scheduler/enqueued': 1,
 'scheduler/enqueued/memory': 1,
 'spider_exceptions/NotImplementedError': 1,
 'start_time': datetime.datetime(2017, 5, 25, 0, 48, 48, 860709)}
2017-05-25 03:48:49 [scrapy.core.engine] INFO: Spider closed (finished)

````

Паук сначала инициализирует и загружает дополнительные компоненты и расширения, которые нужны ему для работы. Затем он переходит по урлу из `start_urls` и загружает HTML. После этого робот передаёт код страницы методу `parse`, который у нас не написан и поэтому ничего не делает. На этом паук заканчивает свою работу. 

У нас есть паук, который открывает страницу, смотрит нанеё и закрывает. Научим нашего паука собирать данные. Напишем метод parse внутри нашего класса. 

In [40]:
class BookSpider(scrapy.Spider):
    name = 'books'
    start_urls = ['http://books.toscrape.com/']
    
    def parse(self, response):
        for href in response.css('.product_pod a::attr(href)').extract():
            url = response.urljoin(href)
            print(url)

Метод `parse` принимает на вход html код страницы и находит в ней нужные нам элементы. Метод `extract()` используется для того, чтобы извлечь из найденного куска разметки сылку. Пример из [документации:](https://doc.scrapy.org/en/latest/intro/tutorial.html#creating-a-project)

```
response.css('title')
[<Selector xpath='descendant-or-self::title'data='<title>Quotes to Scrape</title>'>]
```

```
response.css('title::text').extract()
['Quotes to Scrape']
```


```
response.css('title').extract()
['<title>Quotes to Scrape</title>']
```

Строка `response.urljoin(href)` даёт нам на выход не `catalogue/soumission_998/index.html`, а `http://books.toscrape.com/catalogue/soumission_998/index.html`.

Таким образом код работает следующим образом:

* Паук загружается, переходит по урлу, забирает html.
* Код страницы передаётся методу parse, который осуществляет сбор хрефов.

Попробуем запустить код в консоли и убедиться, что он рабоатет.

In [109]:
# Проверяем код в консоли!!! Можно для этого использовать мэджик...
! scrapy runspider '/home/filipp/Desktop/books/bbk.py'
# Для винды надо использовать cmd!!! 

2017-05-25 05:11:37 [scrapy.utils.log] INFO: Scrapy 1.4.0 started (bot: scrapybot)
2017-05-25 05:11:37 [scrapy.utils.log] INFO: Overridden settings: {'SPIDER_LOADER_WARN_ONLY': True}
2017-05-25 05:11:37 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats',
 'scrapy.extensions.corestats.CoreStats']
2017-05-25 05:11:37 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'scrapy.downloadermiddlewares.redirect

Продолжим дорабатывать код. Добавим ещё один метод. Он будет отвечать за переход по каждой из полученных сылок и сбор данных по ним. Будем собирать названия книг, их цены и изображения. Пройдёмся по всем изменениям, произошедшим с кодом.

`Scrapy` позволяет сохранять собранные данные в видет словарей или csv-файликов. Для того, чтобы проделать это используется ключевое слово `yield`.

In [110]:
class BookSpider(scrapy.Spider):
    name = 'books'
    start_urls = ['http://books.toscrape.com/']
    
    def parse(self, response):
        for href in response.css('.product_pod a::attr(href)').extract():
            url = response.urljoin(href)
            # Сохраним результат применениея функции parse_book к текущему урлу
            yield scrapy.Request(url,self.parse_book)
    
    def parse_book(self,response):
        # Сохраним в виде словаря следующие элементы из респонса:
        yield {
            'title': response.css('h1::text').extract_first(),
            'price': response.css('.prodict_main .price_color::text').extract_first(),
            'image': response.css('#product_gallery ing::attr(src)').extract_first(),
            }
            
# scrapy runspider books1.py -o books.json
# Сохранит книги в формате json

# scrapy runspider books1.py -o books.csv
# Сохранит книги в формате csv

Рассмотрим пример простого парсера. Допустим, мы хотим зайти на сайт habrahabr.ru, считать заголовки последних новостей, далее для каждого заголовка найти картинку с помощью images.yandex.ru и сохранить полученные данные в файл:

In [ ]:
# coding: utf-8
import urllib
import csv
import logging

from grab.spider import Spider, Task

class ExampleSpider(Spider):
    # Список страниц, с которых Spider начнёт работу
    # для каждого адреса в этом списке будет сгенерировано
    # задание с именем initial
    initial_urls = ['http://habrahabr.ru/']

    def prepare(self):
        # Подготовим файл для записи результатов
        # Функция prepare вызываетя один раз перед началом
        # работы парсера
        self.result_file = csv.writer(open('result.txt', 'w'))
        # Этот счётчик будем использовать для нумерации
        # найденных картинок, чтобы создавать им простые имена файлов.
        self.result_counter = 0

    def task_initial(self, grab, task):
        print 'Habrahabr home page'

        # Это функция-обработчик для заданий с именем initial
        # т.е. для тех заданий, что были созданы для
        # адреов указанных в self.initial_urls

        # Как видите интерфейс работы с ответом такой же,
        # как и в обычном Grab
        for elem in grab.xpath_list('//h1[@class="title"]/a[@class="post_title"]'):
            # Для каждой ссылки-заголовка создадим новое задание
            # с именем habrapost
            # Обратите внимание, что мы создаём задания с помощью
            # вызова yield - это сделано исключительно ради красоты
            # По сути, это равносильно следующему коду:
            # self.add_task(Task('habrapost', url=...))
            yield Task('habrapost', url=elem.get('href'))

    def task_habrapost(self, grab, task):
        print 'Habrahabr topic: %s' % task.url

        # Эта функция, как вы уже догадываетесь,
        # получает результаты обработки запросов, которые
        # мы создали для кадого хабратопика, найденного на
        # главной странице хабры

        # Для начала сохраним адрес и заголовк топика в словарь
        post = {
            'url': task.url,
            'title': grab.xpath_text('//h1/span[@class="post_title"]'),
        }

        # Теперь создадим поисковый запрос картинок яндекса, обратите внимание,
        # что мы передаём объекту Task информацию о хабрапосте. Таким образом
        # в функции обработки поиска картинок мы будем знать, для какого именно
        # хабрапоста мы получили результат поиска картинки. Дело в том, что все
        # нестандартные аргументы конструктора Task просто запоминаются в созданном
        # объекте и доступны в дальнейшем как его атрибуты
        query = urllib.quote_plus(post['title'].encode('utf-8'))
        search_url = 'http://images.yandex.ru/yandsearch?text=%s&rpt=image' % query
        yield Task('image_search', url=search_url, post=post)

    def task_image_search(self, grab, task):
        print 'Images search result for %s' % task.post['title']

        # В этой функции мы получили результат обработки поиска картинок, но
        # это ещё не сама картинка! Это только список найденных картинок,
        # Теперь возьмём адрес первой картинки и создадим задание для её
        # скачивания. Не забудем передать информацию о хабрапосте, для которого
        # мы ищем картинку, эта информация хранится в `task.post`.
        image_url = grab.xpath_text('//div[@class="b-image"]/a/img/@src')
        yield Task('image', url=image_url, post=task.post)

    def task_image(self, grab, task):
        print 'Image downloaded for %s' % task.post['title']

        # Это последнняя функция в нашем парсере.
        # Картинка получена, можно сохранить результат.
        path = 'images/%s.jpg' % self.result_counter
        grab.response.save(path)
        self.result_file.writerow([
            task.post['url'].encode('utf-8'),
            task.post['title'].encode('utf-8'),
            path
        ])
        # Не забудем увеличить счётчик ответов, чтобы
        # следующая картинка записалась в другой файл
        self.result_counter += 1


if __name__ == '__main__':
    logging.basicConfig(level=logging.DEBUG)
    # Запустим парсер в многопоточном режиме - два потока
    # Можно больше, только вас яндекс забанит
    # Он вас и с двумя то потоками забанит, если много будете его беспокоить
    bot = ExampleSpider(thread_number=2)
    bot.run()